In [30]:
:dep smartcore = { version = "0.1.0", features=["nalgebra-bindings", "ndarray-bindings", "datasets"]}
:dep nalgebra = "0.22.0"
:dep argmin = "0.3.1"

In [3]:
use nalgebra::{DMatrix, DVector, RowDVector, Scalar};

In [4]:
use std::io::prelude::*;
use std::io::BufReader;
use std::fs::File;
use std::str::FromStr;

In [5]:
fn parse_csv<N, R>(input: R) -> Result<DMatrix<N>, Box<dyn std::error::Error>>
  where N: FromStr + Scalar,
        N::Err: std::error::Error,
        R: BufRead
{
  // initialize an empty vector to fill with numbers
  let mut data = Vec::new();

  // initialize the number of rows to zero; we'll increment this
  // every time we encounter a newline in the input
  let mut rows = 0;

  // for each line in the input,
  for line in input.lines() {
    // increment the number of rows
    rows += 1;
    // iterate over the items in the row, separated by commas
    for datum in line?.split_terminator(",") {
      // trim the whitespace from the item, parse it, and push it to
      // the data array
      data.push(N::from_str(datum.trim())?);
    }
  }

  // The number of items divided by the number of rows equals the
  // number of columns.
  let cols = data.len() / rows;

  // Construct a `DMatrix` from the data in the vector.
  Ok(DMatrix::from_row_slice(rows, cols, &data[..]))
}

In [6]:
let file = File::open("../data/creditcard.csv")?;
let credit: DMatrix<f64> = parse_csv(BufReader::new(file))?;
credit.shape()

(984, 31)

In [7]:
let x = credit.columns(0, 30).into_owned();
let y = credit.column(30).into_owned();
(x.shape(), y.shape())

((984, 30), (984, 1))

In [8]:
use smartcore::model_selection::train_test_split;

In [9]:
let (x_train, x_test, y_train, y_test) = train_test_split(&x, &y.transpose(), 0.2);
(x_train.shape(), y_train.shape(), x_test.shape(), y_test.shape())

((804, 30), (1, 804), (180, 30), (1, 180))

## Sigmoid function

$\sigma(x) = \frac{1}{1 + e^{-x}}$

![title](https://upload.wikimedia.org/wikipedia/commons/8/88/Logistic-curve.svg)

In [32]:
fn sigmoid(v: f64) -> f64 {
    if v < -40. {
        0.
    } else if v > 40. {
        1.
    } else {
        1. / (1. + f64::exp(-v))
    }
}

In [12]:
fn dot(w: &Vec<f64>, x: &DMatrix<f64>, m_row: usize) -> f64 {
    let mut sum = 0f64;
    let (_, p) = x.shape();
    for i in 0..p {
        sum += x[(m_row, i)] * w[i];
    }

    sum + w[p]
}

## Logistic Regression

* Estimated probability (our response is either 1 or 0): $P(X) = \sigma({X}\beta)$
* Cost function: 
$$J(\beta) = -\frac{1}{n}\sum_{i=1}^n[y_i log(\sigma({X}\beta)) + (1- y_i)log(1 - \sigma({X}\beta))]$$
* Derivative: $\frac{\partial J(\beta)}{\partial \beta} = \frac{1}{n} X^T(\sigma(X\beta) - y)$

In [87]:
// y_true = 1.0, y_hat -> +inf
let y = 1.0;
let y_hat = 10.0;
y * sigmoid(y_hat).ln() + (1.0 - y) * (1.0 - sigmoid(y_hat)).ln()

-0.00004539889921682063

In [81]:
// y_true = 1.0, y_hat -> -inf
let y = 1.0;
let y_hat = -10.0;
sigmoid(y_hat).ln() + (1.0 - y) * (1.0 - sigmoid(y_hat)).ln()

-10.000045398899218

In [82]:
// y_true = 0.0, y_hat -> +inf
let y = 0.0;
let y_hat = 10.0;
y * sigmoid(y_hat).ln() + (1.0 - y) * (1.0 - sigmoid(y_hat)).ln()

-10.000045398900186

In [83]:
// y_true = 0.0, y_hat -> -inf
let y = 0.0;
let y_hat = -10.0;
y * sigmoid(y_hat).ln() + (1.0 - y) * (1.0 - sigmoid(y_hat)).ln()

-0.00004539889921682063

In [10]:
use argmin::prelude::*;
use argmin::solver::linesearch::{ArmijoCondition, BacktrackingLineSearch};
use argmin::solver::quasinewton::LBFGS;

In [13]:
struct LossFunction<'a> {
    x: &'a DMatrix<f64>,
    y: &'a DVector<f64>
}

In [14]:
impl<'a> ArgminOp for LossFunction<'a> {
    type Param = Vec<f64>;
    type Output = f64;
    type Hessian = Vec<Vec<f64>>;
    type Jacobian = ();
    type Float = f64;

    // cost function at point w
    fn apply(&self, w: &Self::Param) -> Result<Self::Output, Error> {
        let mut f = 0f64;
        let (n, _) = self.x.shape();

        for i in 0..n {
            let wx = dot(w, &self.x, i);
            f += self.y[i] * sigmoid(wx).ln() + (1.0 - self.y[i]) * (1.0 - sigmoid(wx)).ln();
        }
        
        Ok(-f)
    }

    // the gradient of the cost function at the point w
    fn gradient(&self, w: &Self::Param) -> Result<Self::Param, Error> {
        let (n, p) = self.x.shape();
        let mut g = vec![0f64; w.len()];

        for i in 0..n {
            let wx = dot(w, &self.x, i);

            let dyi = self.y[i] - sigmoid(wx);
            for j in 0..p {
                g[j] -= dyi * self.x[(i, j)];
            }
            g[p] -= dyi;
        }
        Ok(g)
    }    
}

In [15]:
fn optimize(x: &DMatrix<f64>, y: &DVector<f64>) -> Result<(DVector<f64>, f64), Error> {      

    let (_, p) = x.shape();

    // Define cost function
    let cost = LossFunction { x, y };

    // Define initial parameter vector
    let init_param: Vec<f64> = vec![0f64; p + 1];
    
    // Set condition
    let cond = ArmijoCondition::new(0.5)?;

    // set up a line search
    let linesearch = BacktrackingLineSearch::new(cond).rho(0.9)?;

    // Set up solver
    let solver = LBFGS::new(linesearch, 10);

    // Run solver
    let res = Executor::new(cost, solver, init_param)
        .max_iters(100)
        .run()?;

    let w = DVector::from_row_slice(&res.state().best_param);
        
    Ok((w.rows(0, p).into_owned(), w[p]))
}

In [16]:
let (coeff, intercept) = optimize(&x_train, &y_train.transpose()).unwrap();

In [17]:
println!("coeff: {}, intercept: {}", coeff.rows(0, 5), intercept);

coeff: 
  ┌                          ┐
  │ -0.000029086428558675878 │
  │    -0.031008276588418204 │
  │     -0.04175083130759226 │
  │      -0.5853141446539952 │
  │       0.8112769950796751 │
  └                          ┘

, intercept: -0.45322948269666435


In [18]:
fn predict(x: &DMatrix<f64>, coeff: &DVector<f64>, intercept: f64) -> DVector<f64> {
    let mut y_hat = (x * coeff).add_scalar(intercept);
    y_hat.apply(|v| {
        if sigmoid(v) > 0.5 {
            1.0
        } else {
            0.0
        }
    });
    y_hat.into_owned()
}

In [19]:
use smartcore::metrics::roc_auc_score;

In [20]:
roc_auc_score(&y_test, &predict(&x_test, &coeff, intercept).transpose())

0.9255952380952381

In [21]:
use smartcore::linear::logistic_regression::LogisticRegression;

In [22]:
let lr = LogisticRegression::fit(&x_train, &y_train).unwrap();
let y_hat = lr.predict(&x_test).unwrap();

In [23]:
roc_auc_score(&y_test, &y_hat)

0.9315476190476191

In [24]:
println!("y_hat: {}, y_true: {}", y_hat.columns(0, 10), y_test.columns(0, 10));

y_hat: 
  ┌                     ┐
  │ 1 1 0 0 1 1 0 1 0 1 │
  └                     ┘

, y_true: 
  ┌                     ┐
  │ 1 1 0 1 1 1 0 1 0 1 │
  └                     ┘




In [25]:
use smartcore::ensemble::random_forest_classifier::RandomForestClassifier;

In [26]:
let y_hat_rf = RandomForestClassifier::fit(&x_train, &y_train, Default::default()).
            and_then(|rf| rf.predict(&x_test)).unwrap();

In [27]:
roc_auc_score(&y_test, &y_hat_rf)

0.9375